In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import requests
import mwparserfromhell
import urllib.request
import json
import re
import numpy as np

url = "https://wowpedia.fandom.com/wiki/Major_characters"
html = urlopen(url).read()
soup = BeautifulSoup(html, features="html.parser")



In [2]:
characters = []

games = ['WC1','WC2','WC2X','WC3','WC3X','WoW','WoWBC','WoWWotLK','WoWC','WoWMoP','WoWWoD','WoWL','WoWBfA','WoWS','WoWD']

for s in soup.find_all('p'):
    text = s.text
    if "—" in text:
        
        d = {}
        
        t = text.replace("\n", "").split("—")

        name = t[0].strip().replace(" ", "_") #.replace(" ", "")

        expansions = t[1].replace(" ", "").split(",")
        expansions = [1 if i in expansions else 0 for i in games]
        
        if 1 in expansions:
            d['name'] = name
            for i,g in enumerate(games):
                d[g] = expansions[i]
                
            characters.append(d)


In [3]:
df = pd.DataFrame(characters)

In [4]:
names = list(df['name'])
itemus = names[0]

url = f"https://wowpedia.fandom.com/api.php?action=query&prop=revisions&titles={itemus}&rvslots=*&rvprop=content&formatversion=2&format=json"
response = requests.get(url)
 
itemdetail = response.json()
itemdetail['query']['pages'][0]['revisions'][0]['slots']['main']['content']



'{{Foralt}}\n{{npcbox\n| name = Terokk\n| image = Terokk.jpg\n| titles = The Talon King\n| gender = Male\n| level = 72\n| type = Elite\n| race = Arakkoa\n| creature = Undead\n| faction = Combat\n| aggro = {{aggro|-1|-1}}\n| foccupation = Talon King of the Arakkoa, King of the Outcasts and Founder of Skettis\n| status = Deceased (lore)<br/> Killable {{bc-inline}}\n| location = [[Terokk\'s Rest]] in [[Skettis]], [[Terokkar Forest]]\n| relatives = [[Lithic]] (daughter)\n}}\n\n\'\'\'Terokk\'\'\' was the greatest king of the [[arakkoa]]. Terokk was associated with the goddess [[Rukhmar]] and believed to be her incarnation, though this was not actually the case. Terokk was the arakkoa\'s greatest hero who was betrayed by his own allies and afflicted with the Curse of Sethe. Even in this state, however, he sought to aid his fellow refugees by building the city of [[Skettis]] with his own claws. However the curse soon began to drive him to madness, and he was sealed away by his Talonpriests in

In [5]:
t = names[0]

response = requests.get(
     'https://wowpedia.fandom.com/api.php',
         params={
         'action': 'query',
         'format': 'json',
         'titles': '{}'.format(t),
         'prop': 'revisions',
         'rvprop': 'content',
     }
 ).json()


page = next(iter(response['query']['pages'].values()))

wikicode = page['revisions'][0]['*']
parsed_wikicode = mwparserfromhell.parse(wikicode).strip_code()


In [6]:
parsed_wikicode

'Terokk was the greatest king of the arakkoa. Terokk was associated with the goddess Rukhmar and believed to be her incarnation, though this was not actually the case. Terokk was the arakkoa\'s greatest hero who was betrayed by his own allies and afflicted with the Curse of Sethe. Even in this state, however, he sought to aid his fellow refugees by building the city of Skettis with his own claws. However the curse soon began to drive him to madness, and he was sealed away by his Talonpriests in the hopes of rejuvenating him. He now waits in the shadows to be called back into the physical world and face the enemies of his people. \n\nThe heretical Talon King Ikiss claims he is Terokk returned. \n\nTerokk also has an avatar in the form of a gigantic kaliri, unleashed on Shattrath when all of his plans are thwarted by adventurers at the behest of the Skettis refugees of the Lower City. \n\nBiography\nOn Draenor\nthumb|left|Terokk doing battle with the Bloodmane saberon, before becoming cu

In [7]:
baseurl = "https://wowpedia.fandom.com/api.php?"
action = "action=query"
title = "titles={}"
content = "prop=revisions&rvprop=content"
dataformat ="format=json"

query = "{}{}&{}&{}&{}".format(baseurl, action, content, title.format(t), dataformat)
wikiresponse = urllib.request.urlopen(query)
wikidata = wikiresponse.read()
wikitext = wikidata.decode('utf-8')

In [8]:
wikitext

'{"batchcomplete":"","warnings":{"main":{"*":"Subscribe to the mediawiki-api-announce mailing list at <https://lists.wikimedia.org/postorius/lists/mediawiki-api-announce.lists.wikimedia.org/> for notice of API deprecations and breaking changes."},"revisions":{"*":"Because \\"rvslots\\" was not specified, a legacy format has been used for the output. This format is deprecated, and in the future the new format will always be used."}},"query":{"pages":{"86839":{"pageid":86839,"ns":0,"title":"Terokk","revisions":[{"contentformat":"text/x-wiki","contentmodel":"wikitext","*":"{{Foralt}}\\n{{npcbox\\n| name = Terokk\\n| image = Terokk.jpg\\n| titles = The Talon King\\n| gender = Male\\n| level = 72\\n| type = Elite\\n| race = Arakkoa\\n| creature = Undead\\n| faction = Combat\\n| aggro = {{aggro|-1|-1}}\\n| foccupation = Talon King of the Arakkoa, King of the Outcasts and Founder of Skettis\\n| status = Deceased (lore)<br/> Killable {{bc-inline}}\\n| location = [[Terokk\'s Rest]] in [[Skettis

In [9]:
WORD_RE = re.compile(r"\[{2}.*?\]{2}")

for word in WORD_RE.findall(wikitext):
    word = word[2:-2].replace(" ", "_")
    print(word)

Terokk's_Rest
Skettis
Terokkar_Forest
Lithic
arakkoa
Rukhmar
Skettis
Talon_King_Ikiss
kaliri
Lower_City
File:Terrok_Battle.png|thumb|left|Terokk_doing_battle_with_the_Bloodmane_saberon,_before_becoming_cursed.
Image:Avatar_of_Terokk.jpg|thumb|The_Avatar_of_Terokk
Dark_Portal
World_of_Warcraft:_Chronicle_Volume_2
spires_of_Arak
Hidden_in_Plain_Sight
Lithic
Warp_Stalker|stalkers
Wind_Serpent|wind_serpents
Colossus|colossi
Adversarial_Blood
Pridelord_Karash
Bloodmane_tribe|Bloodmane
saberon
Rukhmar
The_Talon_King
Pridelord_Karash
Skyreach
Apexis
Sethekk_Hollow
Sethe
Adherents_of_Rukhmar
Anzu
dread_raven
The_Eye_of_Anzu
Void|dark
arcane
Talonpriest
Skettis
Arakkoa_Outcasts
Talador
Terokkar_Forest
Dreamcatcher
Beakbreaker_of_Terokk
Hazzik's_Bargain
Reshad
Terokk's_Fall
Isfar
Countdown_to_Doom
Veil_Shienor
Veil_Reskk
The_Eyes_of_Skettis
Veil_Skith
Darkstone_of_Terokk
Veil_Skith:_Darkstone_of_Terokk
Veil_Rhaze
Veil_Rhaze:_Unliving_Evil
Veil_Lithic
kaliri
Veil_Lithic:_Preemptive_Strike
Veil_Sh

Below we scape the clean text from each characters page:

In [10]:
from tqdm import tqdm
text_list = []
for name in tqdm(df['name']):
    response = requests.get(
         'https://wowpedia.fandom.com/api.php',
             params={
             'action': 'query',
             'format': 'json',
             'titles': '{}'.format(name),
             'prop': 'revisions',
             'rvprop': 'content',
         }
     ).json()


    page = next(iter(response['query']['pages'].values()))
    try:
        wikicode = page['revisions'][0]['*']
        parsed_wikicode = mwparserfromhell.parse(wikicode).strip_code()
        text_list.append(parsed_wikicode)
    except:
        text_list.append(" ")
    
df.insert(1,'Clean Text',text_list)


100%|████████████████████████████████████████████████████████████████████████████████| 271/271 [02:04<00:00,  2.18it/s]


In [11]:
df_backup = df

In [12]:
from tqdm import tqdm
alphabet = 'abcdefghijklmnopqrstuvwxyz'.upper()
c_list = []
for letter in tqdm(alphabet):
    url = "https://wowpedia.fandom.com/wiki/Category:Lore_characters?pagefrom={}".format(letter)
    html = urlopen(url).read()
    soup = BeautifulSoup(html, features="html.parser")
    
    for i in soup.find_all('li'):
        text = i.find('a')['href']
        verboten = ['/wiki/Genealogy', '/wiki/Local_Sitemap']
        if text[0:5] == '/wiki' and ":" not in text and text not in verboten:
            c_list.append(text.replace("%27", "'").split("/wiki/")[1])

100%|██████████████████████████████████████████████████████████████████████████████████| 26/26 [00:21<00:00,  1.21it/s]


In [13]:
#Find links for all characters:

baseurl = "https://wowpedia.fandom.com/api.php?"
action = "action=query"
title = "titles={}"
content = "prop=revisions&rvprop=content"
dataformat ="format=json"

WORD_RE = re.compile(r"\[{2}.*?\]{2}")

link_list=[]
for name in tqdm(df['name']):
    query = "{}{}&{}&{}&{}".format(baseurl, action, content, title.format(name), dataformat)
    wikiresponse = urllib.request.urlopen(query)
    wikidata = wikiresponse.read()
    wikitext = wikidata.decode('utf-8')
    
    links = []
    for word in WORD_RE.findall(wikitext):
        word = word[2:-2].replace(" ", "_")
        if word in c_list or word in list(df['name']) and word not in links:
            links.append(word)
    link_list.append(np.unique(links))

df.insert(2,'Out Links',link_list)

100%|████████████████████████████████████████████████████████████████████████████████| 271/271 [02:08<00:00,  2.11it/s]


In [14]:
df

,name,Clean Text,Out Links,WC1,WC2,WC2X,WC3,WC3X,WoW,WoWBC,WoWWotLK,WoWC,WoWMoP,WoWWoD,WoWL,WoWBfA,WoWS,WoWD
0,Terokk,Terokk was the greatest king of the arakkoa. T...,"[Anzu, Rukhmar, Sethe, Talon_King_Ikiss]",0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
1,Talon_King_Ikiss,"Talon King Ikiss (pronounced Eye-kiss), the bo...","[Anzu, Sethe, Terokk]",0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
2,Archimonde,Archimonde (pronounced ARK-i-mond) was one of ...,"[Alexstrasza, Azgalor, Cenarius, Grom_Hellscre...",0,0,0,1,0,0,1,1,0,0,1,1,0,0,0
3,Kil'jaeden,Kil'jaeden (sometimes spelled as Kil'Jaeden)Ar...,"[Archimonde, Arthas_Menethil, Garrosh_Hellscre...",0,0,0,0,1,0,1,0,0,0,1,1,0,0,0
4,Mannoroth,Mannoroth was the barbaric leader of the pit l...,"[Archimonde, Cenarius, Durotan, Garrosh_Hellsc...",0,0,0,1,0,0,1,0,1,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
266,Vilnak'dor,Vilnak'dor was a Zandalari troll sent by King ...,"[Khal'ak, Thrall, Vol'jin, Zul]",0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
267,Helya,"Helya is the ruler of Helheim, the underworld ...","[Anduin_Wrynn, Draka, Galakrond, Illidan_Storm...",0,0,0,0,0,0,0,0,0,0,0,1,0,1,0
268,Agatha,Agatha is a Val'kyr in service to the Dark Lad...,"[Annhylde_the_Caller, Sylvanas_Windrunner]",0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
269,Annhylde_the_Caller,thumb|In the Maw.\nthumb|Annhylde in the Sanct...,"[Arthas_Menethil, Bolvar_Fordragon, Helya, Syl...",0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
